# 01 Quickstart / クイックスタート

This notebook demonstrates the end-to-end flow on a synthetic tire. It mirrors the CLI pipeline: load points, fit a cylinder, pick (synthetic) rim points, slice the 12 o'clock band, and output an X–Z' profile.

## Approach / 手法概要
- **Pipeline**: load a GLB or synthetic sample, fit a cylinder, align coordinates, pick rim points, slice the 12 o'clock band, and compute axial (X) vs. rim-zeroed radial (Z') statistics.
- **Coordinates**: after alignment, +X follows the tire axis, +Y is circumferential arc length, and +Z points upward from the rim.
- **Rim-line zero**: rim picks (manual in real runs, synthetic here) regress Z0(Y)=α+βY so that Z' = Z − Z0(Y).

## Cylinder fitting & coordinate alignment / 円筒フィッティングと座標整列
We rely on `pyransac3d.Cylinder.fit`, a RANSAC estimator that handles noisy meshes and outliers. After the cylinder axis is recovered, `align_points` translates the cloud so the fitted axis point becomes the origin, rotates the axis to +X, and finally spins around +X until the highest point lands at +Z (12 o'clock). このノートでは `pyransac3d` のRANSAC円筒フィットを使い、外れ値が多い点群でも安定して軸を推定します。推定後は `align_points` で原点平行移動→軸を +X に回転→ +X 周りに回して 12 時方向が +Z になるように整列します。

## Point-count note / 点群数に関する注意
RANSAC はサンプル数が多いほど時間がかかります。デモでは 60k 点を既定とし、必要に応じて `synthetic_tire_points(n=...)` の引数で減らして下さい。RANSAC 処理は点数が増えると指数的に遅くなるため、Quickstart では 6 万点に抑え、必要に応じて `synthetic_tire_points(n=...)` で変更してください。


In [ ]:
# Ensure the repository root is importable (VS Code / Windows friendly)
import sys
from pathlib import Path

NOTEBOOK_ROOT = Path().resolve()
repo_root = None
for candidate in [NOTEBOOK_ROOT, NOTEBOOK_ROOT.parent, NOTEBOOK_ROOT.parent.parent]:
    if (candidate / "tire_profiler").exists():
        repo_root = candidate
        break

if repo_root is None:
    raise RuntimeError("Unable to locate tire_profiler package relative to this notebook")

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tire_profiler.align import align_points
from tire_profiler.cylinder import fit_cylinder
from tire_profiler.rimline import fit_rimline
from tire_profiler.slice_profile import cylindrical_features, slice_band, compute_profile, plot_profile

In [ ]:
import plotly.graph_objects as go

def preview_cloud(points, color=None, title="Point cloud preview", sample=20000):
    points = np.asarray(points)
    if points.size == 0:
        raise ValueError("No points available for visualization")
    idx = None
    if len(points) > sample:
        rng = np.random.default_rng(0)
        idx = rng.choice(len(points), sample, replace=False)
    pts = points if idx is None else points[idx]
    if color is None:
        color_vals = pts[:, 2]
    else:
        color = np.asarray(color)
        color_vals = color if idx is None else color[idx]
    fig = go.Figure(data=[go.Scatter3d(
        x=pts[:, 0],
        y=pts[:, 1],
        z=pts[:, 2],
        mode='markers',
        marker=dict(size=2, color=color_vals, colorscale='Viridis', opacity=0.85)
    )])
    fig.update_layout(
        title=title,
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='data'),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.show()


In [ ]:
def synthetic_tire_points(seed: int = 0, n: int = 60_000):
    rng = np.random.default_rng(seed)
    x = rng.uniform(-0.12, 0.12, n)
    theta = rng.uniform(-np.pi, np.pi, n)
    radius = 0.34 + 0.002 * np.sin(6 * theta)
    y = radius * np.sin(theta)
    z = radius * np.cos(theta)
    rim_mask = np.abs(x) > 0.09
    z[rim_mask] += 0.006 * np.exp(-((theta[rim_mask]) ** 2) / 0.1)
    noise = rng.normal(scale=0.0008, size=(n, 3))
    pts = np.stack([x, y, z], axis=1) + noise
    return pts

In [ ]:
points = synthetic_tire_points(seed=42)
points.shape

In [ ]:
preview_cloud(
    points,
    title="Raw synthetic tire point cloud / 元の点群プレビュー"
)


## Alignment math / 座標変換の考え方
`align_points` subtracts the fitted axis point to recenter the cloud, applies a rotation that maps the fitted axis vector to +X, and then performs an X-axis rotation so that the global maximum Z point sits at +Z (12 o'clock). In matrix terms we apply \(X' = R_{top} R_{axis} (X - p_{axis})\). `align_points` は推定した軸上の点を原点に平行移動し、軸方向を +X に回す回転行列 \(R_{axis}\) を適用、その後 +X 軸周りの回転 \(R_{top}\) で 12 時方向が +Z になるようにします。\(X' = R_{top} R_{axis} (X - p_{axis})\) という変換を順番に適用しています。


In [ ]:
model = fit_cylinder(points, threshold=0.0025)  # RANSAC cylinder via pyransac3d
aligned, rotation, translation = align_points(points, model.point_on_axis, model.axis_direction)
features = cylindrical_features(aligned, model.radius)
mask = slice_band(aligned, features=features, tape_width=0.02, outer_band=0.05)
print(f"Selected {mask.sum()} points in the 12 o'clock band")
print("Alignment translation (m):", translation)
print("Alignment rotation matrix:\n", np.array_str(rotation, precision=4))

In [ ]:
preview_cloud(
    aligned,
    color=np.where(mask, 1.0, 0.0),
    title="Aligned cloud with 12 o'clock mask / 12時帯域のマスク",
    sample=15000
)


In [ ]:
# Synthetic rim picks near the side wall so the notebook can run non-interactively
side_mask = (np.abs(aligned[:, 0]) > 0.11) & (np.abs(features['arc']) < 0.03)
rim_points = aligned[side_mask][:20]
arc = np.arctan2(rim_points[:, 1], rim_points[:, 2]) * model.radius
rimline = fit_rimline(rim_points, arc)
rimline

In [ ]:
preview_cloud(
    aligned,
    color=np.where(side_mask, 1.0, np.nan),
    title="Rim pick candidate region / リムラインピック候補",
    sample=12000
)


In [ ]:
result = compute_profile(
    aligned,
    features=features,
    mask=mask,
    rimline=rimline,
    nbins=80
)
result.profile.head()

In [ ]:
plot_profile(result.profile, Path('quickstart_profile.png'))
result.profile[['x_center', 'z_mean', 'z_std']].plot(x='x_center', y='z_mean', kind='line')
plt.title("Preview of Z' mean")
plt.show()